# Import libraries

In [1]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd
import pandas_gbq
pd.options.display.max_columns = None
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
import time
import numpy as np
import datetime as dt
import regex as re
sns.set()


# load google biquery and credentials
from google.cloud import bigquery
#client = bigquery.Client()
#%load_ext google.cloud.bigquery

## Get HTML from kleiderkreisel.de search

In [2]:
headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})

In [3]:
url = "https://www.kleiderkreisel.de/kleidung?order=newest_first&search_text=hemd%20minimum%20xl"
response = get(url, headers=headers)

In [4]:
print(response)

<Response [200]>


In [5]:
html_soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
#print(html_soup.prettify())

## Get product detail pages from first page

In [6]:
url = "https://www.kleiderkreisel.de/kleidung?order=newest_first&search_text=hemd%20minimum%20xl"
response = get(url, headers=headers)

html_soup = BeautifulSoup(response.text, 'html.parser')

links = []
for i in html_soup.find_all("div", class_="is-visible item-box__container"):
    for t in i.find_all("section", class_="item-box js-catalog-item"):
        for z in t.find_all("figure", class_="item-box__media media media--item"):
            #print(z.find_all("div", class_="media__placeholder"))
            for y in z.find_all("div", class_="media__placeholder"):
                for u in y.find_all("a", class_="media__image-wrapper js-item-link"):
                    links.append(u.get('href'))

In [7]:
#print(links)

## Get all product lists related to search

In [8]:
next_pages = []
for i in html_soup.find_all("div", class_="new-page-controls js-page-controls box box--plain"):
    for t in i.find_all("li", class_="new-pagination__step"):
        for z in t.find_all("a", class_="new-pagination__action"):
            next_pages.append(z.get('href'))

In [9]:
#next_pages

In [10]:
next_pages_unique = np.unique(next_pages).tolist()[1:]

## Get all product detail pages from all product list pages

In [11]:
for i in next_pages_unique:
    next_url = "https://www.kleiderkreisel.de" + i
    #print(next_url)
    response = get(next_url, headers=headers)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    links_next_page = []
    for i in html_soup.find_all("div", class_="is-visible item-box__container"):
        for t in i.find_all("section", class_="item-box js-catalog-item"):
            for z in t.find_all("figure", class_="item-box__media media media--item"):
                for y in z.find_all("div", class_="media__placeholder"):
                    for u in y.find_all("a", class_="media__image-wrapper js-item-link"):
                        links_next_page.append(u.get('href'))

https://www.kleiderkreisel.de/kleidung?order=newest_first&page=2&search_text=hemd+minimum+xl&time=1556731450


In [12]:
# Concatenate all product detail pages to scrape
product_details = links + links_next_page

In [13]:
product_detail_pages = []
for i in product_details:
    product_url = "https://www.kleiderkreisel.de" + i
    product_detail_pages.append(product_url)
#product_detail_pages

In [14]:
df = pd.DataFrame({'product_detail_pages': product_detail_pages})

In [15]:
#df[df['product_detail_pages']=='https://www.kleiderkreisel.de/herren/hemden-sonstiges/199184821-minimum-hemd-xl']

,product_detail_pages
2,https://www.kleiderkreisel.de/herren/hemden-so...


## Get info from product detail pages

In [16]:
df['date'] = dt.datetime.now().date()
df['product_name'] = np.nan
df['pic_links'] = np.nan
df['price'] = np.nan
df['brand'] = np.nan
df['condition'] = np.nan
df['color'] = np.nan
df['release_datetime'] = np.nan
df['location'] = np.nan
df['user_name'] = np.nan
df['user_last_online'] = np.nan

for index, i in enumerate(product_detail_pages):
    
    # get HTML for each product detail page
    response = get(i, headers=headers)
    #time.sleep(5)
    html_product = BeautifulSoup(response.text, 'html.parser')
    
    # Safe product name
    for i in html_product.find_all("h1", class_="details-list__item-title", itemprop="name"):
        name = i.text
        df['product_name'][index] = name
    
    # Safe thumb nail pics as list to dataframe
    pic_links = []
    for i in html_product.find_all("figure", class_=re.compile("item-description item-photo item-photo--")):
        for u in i.find_all("a", class_=re.compile("item-thumbnail")):
                pic_links.append(u.get('href'))
        df['pic_links'][index] = pic_links
        
    # Safe prices to df
    for i in html_product.find_all("div", class_="c-text--heading c-text--left c-text"):
        price = i.text
        df['price'][index] = price
        
    # Safe brand to df
    for i in html_product.find_all("div", class_="details-list__item-value"):
        for t in i.find_all("span"):
            brand = t.text
            df['brand'][index] = brand
            
    # Safe size to df
    for i in html_product.find_all("div", class_="details-list__item-value", itemprop="itemCondition"):
        condition = i.text
        df['condition'][index] = condition
        
    # Safe color to df
    for i in html_product.find_all("div", class_="details-list__item-value", itemprop="color"):
        color = i.text
        df['color'][index] = color
        
    # Safe release date to df > last online
    datetime = []
    for idf, i in enumerate(html_product.find_all("time", class_="relative")):
        datetime.append(i.get('datetime'))
    df['release_datetime'][index] = datetime[0]
    df['user_last_online'][index] = datetime[1]
    
    # Safe location to df
    # To Do: Get all locations
    for i in html_product.find_all("div", class_="details-list__item details-list--country"):
        location = i.text
        #print(location)
        df['location'][index] = location
    
    # TO DO size, payment options, clicks !!!
    #for i in html_product.find_all("div", class_="details-list__item-value"):
    #    print(i.text)
    
    # Safe user name to df
    for i in html_product.find_all("span", class_="user-login-name"):
        user_name = i.text
        df['user_name'][index] = user_name
    
        
    
    
               

/Users/gunnar.griese/Desktop/env/lib/python2.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/gunnar.griese/Desktop/env/lib/python2.7/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/Users/gunnar.griese/Desktop/env/lib/python2.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/gunnar.griese/Desktop/env/lib/python2.7/si

In [17]:
#df.product_detail_pages[1]

u'https://www.kleiderkreisel.de/herren/karierte-hemden/199515682-minimum-hemd'

In [18]:
df

,product_detail_pages,date,product_name,pic_links,price,brand,condition,color,release_datetime,location,user_name,user_last_online
0,https://www.kleiderkreisel.de/herren/gestreift...,2019-05-01,Minimum Hemd gestreift sommerlich,[https://images.vinted.net/thumbs/f800/0492a_w...,"8,00 €",MINIMUM,Gut,"beige, hellblau",2019-04-28T16:35:42+02:00,"\n Köln, NRW\n",domisolecker,2019-04-30T15:13:55+02:00
1,https://www.kleiderkreisel.de/herren/karierte-...,2019-05-01,Minimum Hemd,[https://images.vinted.net/thumbs/f800/03bf5_m...,"10,00 €",MINIMUM,Gut,"schwarz, weiß",2019-04-21T19:01:33+02:00,"\n Würzburg, BY\n",philipp_hen,2019-05-01T13:20:33+02:00
2,https://www.kleiderkreisel.de/herren/hemden-so...,2019-05-01,Minimum Hemd XL,[https://images.vinted.net/thumbs/f800/03e07_b...,"7,00 €",MINIMUM,Sehr gut,"schwarz, weiß",2019-04-17T13:51:58+02:00,NaN,liber.tine,2019-04-30T23:22:48+02:00
3,https://www.kleiderkreisel.de/herren/einfarbig...,2019-05-01,Olivgrünes minimum Hemd XL,[https://images.vinted.net/thumbs/f800/03218_z...,"20,00 €",MINIMUM,Sehr gut,"grün, khaki",2019-04-16T11:49:56+02:00,NaN,marek189,2019-04-30T19:43:37+02:00
4,https://www.kleiderkreisel.de/herren/hemden-so...,2019-05-01,Camouflage Hemd von minimum,[https://images.vinted.net/thumbs/f800/04678_8...,"15,00 €",MINIMUM,Sehr gut,"braun, grün",2019-03-16T20:30:14+01:00,NaN,sophie190616,2019-04-23T10:49:45+02:00
5,https://www.kleiderkreisel.de/herren/karierte-...,2019-05-01,Karo Hemd,[https://images.vinted.net/thumbs/f800/04053_W...,"4,00 €",MINIMUM,Sehr gut,"beige, blau",2019-03-10T13:48:12+01:00,NaN,da1987ni,2019-04-30T19:47:57+02:00
6,https://www.kleiderkreisel.de/herren/karierte-...,2019-05-01,Knitterhemd Sommer XL,[https://images.vinted.net/thumbs/f800/03401_s...,"10,00 €",MINIMUM,Sehr gut,"blau, weiß",2019-03-09T18:27:30+01:00,NaN,tomosterw,2019-04-07T10:53:05+02:00
7,https://www.kleiderkreisel.de/herren/einfarbig...,2019-05-01,Minimum Cord Hemd - XL - Braun - Holzknöpfe - ...,[https://images.vinted.net/thumbs/f800/r2/02ca...,"10,00 €",MINIMUM,Gut,braun,2019-03-05T10:09:17+01:00,NaN,nealcassady83,2019-05-01T19:00:45+02:00
8,https://www.kleiderkreisel.de/herren/hemden-so...,2019-05-01,Lässiges Minimum Hemd,[https://images.vinted.net/thumbs/f800/03154_3...,"12,00 €",MINIMUM,Sehr gut,blau,2019-02-17T18:20:10+01:00,"\n Magdeburg, SA\n",maxmd,2019-04-19T10:20:24+02:00
9,https://www.kleiderkreisel.de/damen/langarmeli...,2019-05-01,Rotes Hemd Boyfriendstil von minimum XL,[https://images.vinted.net/thumbs/f800/04db3_Y...,"7,50 €",MINIMUM,Sehr gut,burgunderrot,2019-02-11T16:13:33+01:00,NaN,kk-mitglied11,2019-05-01T11:50:45+02:00


In [19]:
# Authenticate local machine to GCP
!gcloud auth application-default login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&prompt=select_account&response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform&access_type=offline



Credentials saved to file: [/Users/gunnar.griese/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests
Application Default Credentials.

To generate an access token for other uses, run:
  gcloud auth application-default print-access-token


In [20]:
project = 'kleiderkreisel-scraper'
dataset_table_id = 'gunnar.minimum_shirt'
client = bigquery.Client()

/Users/gunnar.griese/Desktop/env/lib/python2.7/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [21]:
pandas_gbq.to_gbq(df, dataset_table_id, project, if_exists='append')

1it [00:18, 18.53s/it]
